## Common Loop Tricks

### Looping a given number of times with *range()*

Loop 5 times using **while**

In [ ]:
i=0
while i < 5:
    print i
    i += 1

Loop 5 times using **for** and **range()**

In [ ]:
# This is way better than the while method, above.

for x in range(5):
    print x

What is **range** doing? The **range** function returns another type of **iterable**. As you can see, *range(x)* returns an iterable that contains all the integers from 0 to *x* in order. You don't even have to use these integers, which is handy when you just want to run some code *x* times.

In [ ]:
for x in range(5):
    print 'hello'

This is a good moment to practice reading the Python documentation. Go [here](https://docs.python.org/2/library/functions.html#range) and read the entry on the **range** function. You may not be able to fully understand it yet, but hopefully you can see that **range** has a couple optional arguments that let you count from numbers other than 0, and by numbers other than 1.

In [ ]:
for x in range(5, 12, 2):
    print x

You can even count backwards by making *step* negative.

In [ ]:
for x in range(12, 5, -2):
    print x

### Looping with *enumerate()*

Lets say we have a sequence with some Ns in it, and we want to find the positions of all the Ns within the sequence. We can use a **for** loop similar to how we did above to find the GC content of a sequence, but this time we will need to remember the indexes of the bases as we loop through the sequence.

Instead of looping through all the bases themselves, we actually want to loop through the indexes of the bases (since we can use these indexes to access the bases themselves). We can generate all the indexes by combining the **range** and **len** functions

In [ ]:
sequence = "ATCGATNGCTANCGTAGCNT"

print range(len(sequence))

for i in range(len(sequence)):
    if sequence[i] == "N":
        print i

This kind of problem is so common, in fact, that Python has a built-in alternative to make this prettier and faster, **enumerate()**.

**enumerate()** takes an **iterable** as an argument and returns another **iterable** with both the index of the item as well as the item itself. Python is actually able to assign values to more than one variable at once (thanks to something called "tuples" that we'll learn about later) so you can capture both outputs of **enumerate** in two variables. To see how this works, try:

In [ ]:
x, y, z = 10, 15, -3

print x
print y
print z

In [ ]:
# *prettier* method using enumerate()
seq = "ATCGATNGCTANCGTAGCNT"

for i, base in enumerate(seq):
    if base == "N":
        print i

## Escaping loops

Occasionally, you might want to get out of a loop before the truth statement is met (with a **while** loop) or you've gone through every element (with a **for** loop). You can modify the default flow of the loop using **break** and **continue**. The keyword **break** ends the loop right where you are, while the keyword **continue** goes back to the top of the loop (bringing in the next object from the **iterable** if it's a **for** loop).

In [ ]:
while True:
    break
    print 'this will not print'
    
for x in range(2):
    print 'this will print twice'
    continue
    print 'this will not print'

In [ ]:
# A calculator program to tell you if numbers are prime.

while True:
    number = raw_input("Number to test: ")
    
    # Quit if nothing is entered.
    if number == '':
        break
    
    number = int(number)
    
    # Prime numbers are >1 by definition.
    # If a number <= 1 is entered, stop and start over.
    if number <= 1:
        print 'Please enter a number greater than 1'
        continue
    
    prime = True
    for x in range(2, number):
        # Use modulo to test if x is a divisor of number
        # if so, the number is not prime, stop the search
        if number % x == 0:
            print 'Not prime,', x, 'is a factor'
            prime = False
            break
    
    if prime:
        print number, 'is prime!'

In this second example there are two loops. The top **while** loop will run until the user enters a blank input, but will otherwise constantly ask the user for numbers to test. If the number entered is <=1, we don't even bother checking for divisors and the loop goes back to the **while** logical expression.

In the second loop, if the user enters a number >1, we assume the number is prime, then check every integer between 2 and *number* to see if it's a divisor. If we find a divisor, we know that *number* is not prime, so set *prime* to **False**, then use **break** to stop checking the rest of the integers. Lastly, if *prime* is still set to **True**, we report that the number is prime.